In [1]:
import gurobipy

Cities = [("A", "B"), ("A", "C"), ("B", "C"), ("B", "D"), ("C", "D")]
Routes = gurobipy.tuplelist(Cities)
Routes.select("A", "*")

<gurobi.tuplelist (2 tuples, 2 values each):
 ( A , B )
 ( A , C )
>

In [1]:
import gurobipy as grb

al = grb.tuplelist([(1, 2), (1, 3), (2, 4)])
bl = grb.tuplelist([("A", "B", "C"), ("A", "C", "D")])

In [2]:
print(al.select(1, "*"))
print(al.select([1, 2]))

<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
>
<gurobi.tuplelist (3 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 2 , 4 )
>


In [59]:
print(bl.select("*", "B", "*"))  # 只要中间是 "B" 就满足

<gurobi.tuplelist (1 tuples, 3 values each):
 ( A , B , C )
>


In [3]:
d = grb.tupledict([((1, 2), "onetwo"), ((1, 3), "onethree"), ((2, 3), "twothree")])
print(d[1, 2])  # prints 'onetwo'

onetwo


In [62]:
print(d.select())  # prints ['onetwo', 'onethree', 'twothree']
print(d.select(1, "*"))  # prints ['onetwo', 'onethree']
print(d.select("*", 3))  # prints ['onethree', 'twothree']
print(d.select(1, 3))  # prints ['onethree']

['onetwo', 'onethree', 'twothree']
['onetwo', 'onethree']
['onethree', 'twothree']
['onethree']


In [ ]:
model = grb.Model(name="test")
# 创建稀疏变量集合,只在一些需要的位置创建求解变量
x = model.addVars([(1, 2), (1, 3), (2, 3)], name="x")
expr1 = x.sum()  # LinExpr: x[1,2] + x[1,3] + x[2,3]
expr2 = x.sum(1, "*")  # LinExpr: x[1,2] + x[1,3]
expr3 = x.sum("*", 3)  # LinExpr: x[1,3] + x[2,3]
expr4 = x.sum(1, 3)  # LinExpr: x[1,3]

model.update()
print(expr1)
print(expr2)
print(expr3)
print(expr4)

x[1,2] + x[1,3] + x[2,3]
x[1,2] + x[1,3]
x[1,3] + x[2,3]
x[1,3]


In [70]:
model = grb.Model(name="test")

# 使用多个整数创建变量
x = model.addVars(3, 4, name="x")  # 创建 3×4 的变量矩阵

# 使用 sum() 方法的示例
expr1 = x.sum()  # 所有变量求和: x[0,0] + x[0,1] + ... + x[2,3]
expr2 = x.sum(1, "*")  # 第一维固定为1: x[1,0] + x[1,1] + x[1,2] + x[1,3]
expr3 = x.sum("*", 2)  # 第二维固定为2: x[0,2] + x[1,2] + x[2,2]
expr4 = x.sum(0, 1)  # 特定索引: x[0,1]

model.update()
print(expr1)
print(expr2)
print(expr3)
print(expr4)

x[0,0] + x[0,1] + x[0,2] + x[0,3] + x[1,0] + x[1,1] + x[1,2] + x[1,3] + x[2,0] + x[2,1] + x[2,2] + x[2,3]
x[1,0] + x[1,1] + x[1,2] + x[1,3]
x[0,2] + x[1,2] + x[2,2]
x[0,1]


In [ ]:
model = grb.Model(name="test")

# 使用单个个整数创建变量
x = model.addVars(3, name="x")  # 创建 3 个变量向量

model.update()
x.sum()

<gurobi.LinExpr: x[0] + x[1] + x[2]>

In [79]:
from gurobipy import quicksum, GRB
import pandas as pd

# Define supplier and customer names
suppliers = ["Supplier_A", "Supplier_B", "Supplier_C"]
customers = ["Customer_1", "Customer_2", "Customer_3", "Customer_4"]

# Define cost matrix
cost_dict = {
    ("Supplier_A", "Customer_1"): 5,
    ("Supplier_A", "Customer_2"): 2,
    ("Supplier_A", "Customer_3"): 6,
    ("Supplier_A", "Customer_4"): 7,
    ("Supplier_B", "Customer_1"): 3,
    ("Supplier_B", "Customer_2"): 5,
    ("Supplier_B", "Customer_3"): 4,
    ("Supplier_B", "Customer_4"): 6,
    ("Supplier_C", "Customer_1"): 4,
    ("Supplier_C", "Customer_2"): 5,
    ("Supplier_C", "Customer_3"): 2,
    ("Supplier_C", "Customer_4"): 3,
}

# Supply and demand quantities
supply = {"Supplier_A": 300, "Supplier_B": 200, "Supplier_C": 400}
demand = {"Customer_1": 200, "Customer_2": 100, "Customer_3": 450, "Customer_4": 250}

# Create model
model = grb.Model(name="Named Transportation Problem")
model.Params.OutputFlag = 0

# Create variables - using string indices
x = model.addVars(
    suppliers, customers, name="transport", vtype=GRB.INTEGER, lb=30, ub=500
)

# Objective function
model.setObjective(
    quicksum(cost_dict[i, j] * x[i, j] for i in suppliers for j in customers),
    sense=GRB.MINIMIZE,
)

# Supply constraints - demonstrate sum() method filtering by name
for s in suppliers:
    # x.sum(s, "*") sums the transport quantities from that supplier to all customers
    model.addConstr(x.sum(s, "*") == supply[s], name=f"Supply_{s}")

# Demand constraints - demonstrate sum() method filtering by name
for c in customers:
    # x.sum("*", c) sums the transport quantities from all suppliers to that customer
    model.addConstr(x.sum("*", c) <= demand[c], name=f"Demand_{c}")

# Solve the model
model.optimize()

# Display sum() method filtering functionality
print("=== tupledict.sum() method examples ===")
print(f"Total transport quantity: {x.sum()}")
print(f"Total transport from Supplier_A: {x.sum('Supplier_A', '*')}")
print(f"Total received by Customer_1: {x.sum('*', 'Customer_1')}")
print(f"Transport from Supplier_B to Customer_3: {x.sum('Supplier_B', 'Customer_3')}")
print()


=== tupledict.sum() method examples ===
Total transport quantity: transport[Supplier_A,Customer_1] + transport[Supplier_A,Customer_2] + transport[Supplier_A,Customer_3] + transport[Supplier_A,Customer_4] + transport[Supplier_B,Customer_1] + transport[Supplier_B,Customer_2] + transport[Supplier_B,Customer_3] + transport[Supplier_B,Customer_4] + transport[Supplier_C,Customer_1] + transport[Supplier_C,Customer_2] + transport[Supplier_C,Customer_3] + transport[Supplier_C,Customer_4]
Total transport from Supplier_A: transport[Supplier_A,Customer_1] + transport[Supplier_A,Customer_2] + transport[Supplier_A,Customer_3] + transport[Supplier_A,Customer_4]
Total received by Customer_1: transport[Supplier_A,Customer_1] + transport[Supplier_B,Customer_1] + transport[Supplier_C,Customer_1]
Transport from Supplier_B to Customer_3: transport[Supplier_B,Customer_3]



In [80]:
# Display results in DataFrame format
if model.status == GRB.OPTIMAL:
    print("=== Optimal Solution ===")
    print(f"Minimum total cost: {model.objVal}")
    print()

    # Create results matrix
    results = []
    for s in suppliers:
        row = {"Supplier": s}
        for c in customers:
            row[c] = x[s, c].X  # .X gets the optimal value of the variable
        row["Total Supply"] = sum(x[s, c].X for c in customers)
        results.append(row)

    # Add demand row
    demand_row = {"Supplier": "Total Demand"}
    for c in customers:
        demand_row[c] = sum(x[s, c].X for s in suppliers)
    results.append(demand_row)

    # Create DataFrame
    df = pd.DataFrame(results)
    print(df.to_string(index=False))
else:
    print("No optimal solution found")

=== Optimal Solution ===
Minimum total cost: 3430.0

    Supplier  Customer_1  Customer_2  Customer_3  Customer_4  Total Supply
  Supplier_A        60.0        40.0       110.0        90.0         300.0
  Supplier_B       110.0        30.0        30.0        30.0         200.0
  Supplier_C        30.0        30.0       310.0        30.0         400.0
Total Demand       200.0       100.0       450.0       150.0           NaN


In [81]:
results

[{'Supplier': 'Supplier_A',
  'Customer_1': 60.0,
  'Customer_2': 40.0,
  'Customer_3': 110.0,
  'Customer_4': 90.0,
  'Total Supply': 300.0},
 {'Supplier': 'Supplier_B',
  'Customer_1': 110.0,
  'Customer_2': 30.0,
  'Customer_3': 30.0,
  'Customer_4': 30.0,
  'Total Supply': 200.0},
 {'Supplier': 'Supplier_C',
  'Customer_1': 30.0,
  'Customer_2': 30.0,
  'Customer_3': 310.0,
  'Customer_4': 30.0,
  'Total Supply': 400.0},
 {'Supplier': 'Total Demand',
  'Customer_1': 200.0,
  'Customer_2': 100.0,
  'Customer_3': 450.0,
  'Customer_4': 150.0}]

In [ ]:
a = {"a": 1, "b": 2}
b = {"a": 3, "b": 4}
data = [a, b]
df = pd.DataFrame(data)  # 以行为视角创建
df


,a,b
0,1,2
1,3,4


In [ ]:
data = {"a": [1, 3], "b": [2, 4]}
df = pd.DataFrame(data)  # 以列为视角创建
df

,a,b
0,1,2
1,3,4


In [78]:
model.getVars()

[<gurobi.Var transport[Supplier_A,Customer_1] (value 60.0)>,
 <gurobi.Var transport[Supplier_A,Customer_2] (value 40.0)>,
 <gurobi.Var transport[Supplier_A,Customer_3] (value 110.0)>,
 <gurobi.Var transport[Supplier_A,Customer_4] (value 90.0)>,
 <gurobi.Var transport[Supplier_B,Customer_1] (value 110.0)>,
 <gurobi.Var transport[Supplier_B,Customer_2] (value 30.0)>,
 <gurobi.Var transport[Supplier_B,Customer_3] (value 30.0)>,
 <gurobi.Var transport[Supplier_B,Customer_4] (value 30.0)>,
 <gurobi.Var transport[Supplier_C,Customer_1] (value 30.0)>,
 <gurobi.Var transport[Supplier_C,Customer_2] (value 30.0)>,
 <gurobi.Var transport[Supplier_C,Customer_3] (value 310.0)>,
 <gurobi.Var transport[Supplier_C,Customer_4] (value 30.0)>]

查看模型中的约束条件或者变量，以确定是否和自己定义的模型一致


In [5]:
# 建立模型
import gurobipy as grb
from gurobipy import GRB, quicksum

cost = [[5, 2, 6, 7], [3, 5, 4, 6], [4, 5, 2, 3]]
demand_num = [200, 100, 450, 250]
product_num = [300, 200, 400]

# 定义模型
model = grb.Model(name="Minimize transfer cost")

# 设置模型参数
model.Params.OutputFlag = 0

# 定义求解变量
transfer_num = model.addVars(
    3, 4, name="transfer_num", vtype=GRB.INTEGER, lb=30, ub=500
)

# 给出目标函数
model.setObjective(
    quicksum(cost[i][j] * transfer_num[i, j] for i in range(3) for j in range(4)),
    sense=GRB.MINIMIZE,
)

# 给出约束条件
for row in range(3):
    model.addConstr(
        grb.quicksum(transfer_num[row, col] for col in range(4)) == product_num[row],
        name="Product Num constrain",
    )

for col in range(4):
    model.addConstr(
        grb.quicksum(transfer_num[row, col] for row in range(3)) <= demand_num[col],
        name="Demand Num constrain",
    )


# 我现在不想求解，而是查看模型现有的内容，以确定约束和目标是否是我期望的形式

用 `m.write()` 导出模型文件 `.lp` 是可直接理解的形式


In [6]:
model.write("debug.lp")
print("输出成功")

输出成功


获取特定的信息，而不是打印整个模型配置信息


In [ ]:
# 查看目标
model.getObjective()

<gurobi.LinExpr: 5.0 transfer_num[0,0] + 2.0 transfer_num[0,1] + 6.0 transfer_num[0,2] + 7.0 transfer_num[0,3] + 3.0 transfer_num[1,0] + 5.0 transfer_num[1,1] + 4.0 transfer_num[1,2] + 6.0 transfer_num[1,3] + 4.0 transfer_num[2,0] + 5.0 transfer_num[2,1] + 2.0 transfer_num[2,2] + 3.0 transfer_num[2,3]>

In [41]:
# 查看约束
print(model.getRow(model.getConstrs()[0]))  # 输出第一个约束对象
print(model.getConstrs()[0].RHS)
print(model.getConstrs()[0].Sense)
print(model.getConstrs()[0].ConstrName)

transfer_num[0,0] + transfer_num[0,1] + transfer_num[0,2] + transfer_num[0,3]
300.0
=
Product Num constrain


In [44]:
model.getVars()[0].VarName

'transfer_num[0,0]'

In [45]:
model.getVarByName("transfer_num[0,0]")

<gurobi.Var transfer_num[0,0]>

查看模型的变量数、约束数等


In [49]:
print(model.NumConstrs)
print(model.NumVars)
print(model.NumObj)
print(model.Status)

7
12
1
1
